# Evaluation: Grammar Miner

* Code for subjects [here](#Our-subject-programs)
* Evaluation starts [here](#Evaluation)
* The evaluation on specific subjects starts [here](#Subjects)
    * [CGIDecode](#CGIDecode)
    * [Calculator](#Calculator)
    * [MathExpr](#MathExpr)
    * [URLParse](#URLParse)
    * [Netrc](#Netrc)
    * [Microjson](#Microjson)
* Results are [here](#Results)

### Accuracy

Our first question concerns the accuracy of the generated grammars which means, does our mined grammars from both techniques represent strings that would be rejected by our target program? In order to answer this question, we use the grammars produced by the two techniques as *producers*. that is, we start with the start symbol and continuously expand nonterminals according to grammar rules, then the resulting string would be fed into the target program which would either accept or reject it.

### Completeness

The next question concerns the completeness of the generated grammars which basically means to what extent does the generated grammars not contain strings that would be accepted by the test subject? In order to verify this, we used a reference grammar as a producer which would create arbitrary strings which would then be parsed by the generated grammars. 
A 100% completeness indicates that the mined grammar holds all inputs of the reference grammar.

In [33]:
import fuzzingbook_utils

## Reference Grammar

In [34]:
REFERENCE_URL_GRAMMAR = {
    "<start>" : [
        "<url>"
    ],
    "<url>" : [
      "<scheme>://<authority><path><query>"  
    ],
    "<scheme>": [
        "http",
        "https",
        "ftp",
        "ftps"      
    ],
    "<authority>": [
        "<host>",
        "<host>:<port>",
        "<userinfo>@<host>",
        "<userinfo>@<host>:<port>"
    ],
    "<user>": [
        "user1",
        "user2",
        "user3",
        "user4",
        "user5"
    ],
    "<pass>": [
        "pass1",
        "pass2",
        "pass3",
        "pass4",
        "pass5"
    ],
    "<host>": [
        "host1",
        "host2",
        "host3",
        "host4",
        "host5"
    ],
    "<port>": [
        "<nat>"
    ],
    "<nat>": [
        "10",
        "20",
        "30",
        "40",
        "50"
    ],
    "<userinfo>": [
        "<user>:<pass>"
    ],
    "<path>": [
        "",
        "/",
        "/<id>",
        "/<id><path>"
    ],
    "<id>": [
        "folder"
    ],
    "<query>": [
        "",
        "?<params>"
    ],
    "<params>": [
        "<param>",
        "<param>&<params>"
    ],
    "<param>": [
        "<key>=<value>"
    ],
    "<key>": [
        "key1",
        "key2",
        "key3",
        "key4"
    ],
    "<value>": [
        "value1",
        "value2",
        "value3",
        "value4"
    ]
}

## Grammar Miner Experiment

In [35]:
from GrammarCoverageFuzzer import GrammarCoverageFuzzer

In [37]:
class GrammarMinerExperiment:
    def __init__(self, reference_grammar, target, **kwargs):
        self.options(kwargs)
        self.grammar = reference_grammar
        self.target_program = target
        self.max_no_samples = 1000
        
        self.gcf = GrammarCoverageFuzzer(self.grammar)
        self.generated_samples = self.generate_sample_inputs() 

In [38]:
class GrammarMinerExperiment(GrammarMinerExperiment):
    def options(self, kwargs):
        self.files = kwargs.get('files', [])
        self.methods = kwargs.get('methods', [])

In [40]:
class GrammarMinerExperiment(GrammarMinerExperiment):
    def generate_sample_inputs(self):
        return [self.gcf.fuzz() for _ in range(self.max_no_samples)]

In [41]:
from GrammarMiner import recover_grammar, recover_grammar_with_taints, syntax_diagram

In [42]:
class GrammarMinerExperiment(GrammarMinerExperiment):
    def mine_with_taint(self):
        grammar_from_taint = recover_grammar_with_taints(
            self.target_program,
            self.generated_samples,
            files=self.files,
            methods=self.methods)

        return grammar_from_taint

In [43]:
class GrammarMinerExperiment(GrammarMinerExperiment):
    def mine_with_substr_search(self):
        grammar_from_substr = recover_grammar(self.target_program,
                                              self.generated_samples,
                                              files=self.files,
                                              methods=self.methods)
        return grammar_from_substr

In [44]:
from urllib.parse import urlparse

In [45]:
ex = GrammarMinerExperiment(REFERENCE_URL_GRAMMAR,
                            urlparse,
                            methods=['urlparse'],
                            files=['urllib/parse.py'])

grammar_from_taint = ex.mine_with_taint()
grammar_from_str = ex.mine_with_substr_search()

In [46]:
grammar_from_taint

{'<start>': ['<urlparse@361:url>'],
 '<urlparse@361:url>': ['http://<urlparse@369:netloc><urlparse@369:url>?<urlparse@369:query>',
  'http://<urlparse@369:netloc><urlparse@369:url>',
  '<urlparse@369:scheme>://<urlparse@369:netloc>',
  '<urlparse@369:scheme>://<urlparse@369:netloc><urlparse@369:url>?<urlparse@369:query>',
  '<urlparse@369:scheme>://<urlparse@369:netloc><urlparse@369:url>',
  'http://<urlparse@369:netloc>?<urlparse@369:query>',
  'http://<urlparse@369:netloc>',
  '<urlparse@369:scheme>://<urlparse@369:netloc>?<urlparse@369:query>'],
 '<urlparse@369:scheme>': ['ftps', 'https', 'ftp'],
 '<urlparse@369:netloc>': ['user5:pass4@host1',
  'user4:pass1@host1',
  'host1:40',
  'user4:pass2@host4:10',
  'user2:pass5@host4:50',
  'user5:pass1@host4:50',
  'user5:pass3@host1',
  'user5:pass2@host5:20',
  'user5:pass4@host1:40',
  'user3:pass2@host1',
  'user2:pass5@host3',
  'user3:pass3@host2:10',
  'user1:pass4@host2:20',
  'user1:pass1@host2:30',
  'user1:pass2@host3:40',
  'us

## Grammar Miner Evaluator

In [47]:
class GrammarMinerEvaluator:
    def __init__(self, target, r_grammar):
        self.max_no_inputs = 1000
        self.target_program = target
        self.grammar = r_grammar

        self.gcf = self.create_gcf_instance(self.grammar)
        self.samples = [self.gcf.fuzz() for i in range(self.max_no_inputs)]

In [12]:
class GrammarMinerEvaluator(GrammarMinerEvaluator):
    def create_gcf_instance(self, grammar):
        return GrammarCoverageFuzzer(grammar)

In [13]:
class GrammarMinerEvaluator(GrammarMinerEvaluator):
    def url_parser_accuracy_test(self, mined_grammar):
        gcf = self.create_gcf_instance(mined_grammar)
        schemes = ['ftps', 'http', 'https', 'ftp']
        rejected = []

        for i in range(self.max_no_inputs):
            s = gcf.fuzz()
            parsed_url = urlparse(s)
            if parsed_url.scheme not in schemes or not bool(parsed_url.netloc):
                rejected.append(s)
        return (((self.max_no_inputs - len(rejected)) / self.max_no_inputs) *
                100)

In [14]:
from Parser import EarleyParser

In [15]:
class GrammarMinerEvaluator(GrammarMinerEvaluator):
    def completeness_test(self, mined_grammar):
        rejected = []
        parser = EarleyParser(mined_grammar)
        for url in self.samples:
            try:
                tree, *_ = parser.parse(url)
            except SyntaxError:
                rejected.append(url)
        return (((self.max_no_inputs - len(rejected)) / self.max_no_inputs) *
                100)

In [16]:
results = {}
ev = GrammarMinerEvaluator(urlparse, REFERENCE_URL_GRAMMAR)

In [ ]:
taint_accuracy = ev.url_parser_accuracy_test(grammar_from_taint)
substr_search_accuracy = ev.url_parser_accuracy_test(grammar_from_str)

In [ ]:
taint_completeness = ev.completeness_test(grammar_from_taint)
substr_search_completeness = ev.completeness_test(grammar_from_str)

In [ ]:
results['Grammar_For_String_Inclusion'] = (substr_search_accuracy, substr_search_completeness)
results['Grammar_From_Taint'] = (taint_accuracy, taint_completeness)

## MicroJson

In [17]:
import microjson

In [18]:
from GrammarMiner import collapse_rules

In [20]:
json_samples = [i.strip().replace('\n', ' ') for i in ['''
{"abcd":[],
  "efgh":{"y":[],
    "pqrstuv":  null,
    "p":  "",
    "q":"" ,
    "r": "" ,
    "float1": 1.0,
    "float2":1.0,
    "float3":1.0 ,
    "float4": 1.0 ,
     "_124": {"wx" :  null,
     "zzyym!!2@@39": [1.1, 2452, 398, {"x":[[4,53,6,[7  ,8,90 ],10]]}]} }
 }
''']]

In [27]:
def convert_spaces(grammar):
    keys = {key: key.replace(' ', '_') for key in grammar}
    new_grammar = {}
    for key in grammar:
        new_alt = []
        for rule in grammar[key]:
            new_rule = []
            for t in rule:
                for k in keys:
                    t = t.replace(k, keys[k])
                new_rule.append(t)
            new_alt.append(''.join(new_rule))
        new_grammar[keys[key]] = new_alt
    return new_grammar

In [32]:
grammar = recover_grammar(microjson.main, json_samples)
grammar

{'<start>': ['<tell@114:self.buf>'],
 '<tell@114:self.buf>': ['{"abcd":[],   "efgh":{"y":[],     "pqrstuv":  <from_json@312:v.efgh._124.wx>,     "p":  "",     "q":"" ,     "r": "" ,     "float1": <from_json@312:v.efgh.float4>,     "float2":1.0,     "float3":1.0 ,     "float4": 1.0 ,      "_124": {"wx" :  null,      "zzyym!!2@@39": [1.1, 2452, 398, {"x":[[4,53,6,[7  ,8,90 ],10]]}]} }  }'],
 '<from_json@312:v.efgh._124.wx>': ['null'],
 '<from_json@312:v.efgh.float4>': ['1.0']}

## Results

In [ ]:
from IPython.display import HTML, display

In [ ]:
def show_table(keys, a, c, title):
    keys = [k for k in keys if k in a and k in c and a[k] and c[k]]
    tbl = ['<tr>%s</tr>' % ''.join(["<th>%s</th>" % k for k in ['<b>%s</b>' % title, 'Accuracy (%)', 'Completeness (%)']])]
    for k in keys:
        h_c = "<td>%s</td>" % k
        a_c = "<td>%s</td>" % a.get(k, ('', 0))[0]
        m_c = "<td>%s</td>" % c.get(k, ('', 0))[1]
        tbl.append('<tr>%s</tr>' % ''.join([h_c, a_c, m_c]))
    return display(HTML('<table>%s</table>' % '\n'.join(tbl)))

In [ ]:
show_table(results.keys(), results, results, 'Grammar Mining Techniques')

In the experiment demonstrated above, the generated grammar both techniques proves to be accurate, out of 1000 urls it produces, all were valid urls which are been accepted by the test subject. Also, in terms of completeness, the grammars produced from both techniques could only parse about 60% (which is 608 urls out of 1000) of the samples generated by the reference grammar this is as a result of the grammar being too specific which means it could only parse similar strings which it can produce itself.